In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
import pandas as pd

In [3]:
spark = (
    SparkSession.builder.
    appName("Working with tabular data").
    getOrCreate())

In [4]:
PATH = r"C:\Users\oluwa\Documents\broadcast_logs"

In [5]:
logs = spark.read.csv(os.path.join(PATH, 'BroadcastLogs_2018_Q3_M8.csv'),
                      header=True,
                      sep="|",
                      inferSchema=True,
                      timestampFormat="yyyy-MM-dd"
                     )

In [ ]:
logs.printSchema()

In [ ]:
logs.columns

In [ ]:
logs.select('BroadcastLogID',
 'LogServiceID',
 'LogDate').show(5, truncate=False)

In [6]:
#exclude 'BroadcastLogID' and 'SequenceNO' columns

logs_copy = logs.select(*[x for x in logs.columns if x not in ['BroadcastLogID', 'SequenceNO', 'LogServiceID']])

In [ ]:
logs_copy.columns

In [ ]:
logs_copy.select(F.col('LogDate')).show(5, False)

In [ ]:
logs_copy.select(F.substring(F.col('LogDate'), 1, 4)).distinct().show(5)

In [ ]:
logs_copy.select(
    F.col('Duration').alias("full_duration")).show(5, False)

In [7]:
logs_copy.select(
    F.col('Duration').alias('full'),
    F.col('Duration').substr(1,2).cast('int').alias('hour'),
    F.col('Duration').substr(4,2).cast('int').alias('min'),
    F.col('Duration').substr(7,2).cast('int').alias('sec'),
    (F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
    F.col('Duration').substr(4,2).cast('int') * 60 +
    F.col('Duration').substr(7,2).cast('int')).alias('duration_secs')
).distinct().show(5, False)

+----------------+----+---+---+-------------+
|full            |hour|min|sec|duration_secs|
+----------------+----+---+---+-------------+
|00:59:52.0000000|0   |59 |52 |3592         |
|00:16:31.0000000|0   |16 |31 |991          |
|00:13:30.0000000|0   |13 |30 |810          |
|00:48:17.0000000|0   |48 |17 |2897         |
|00:00:19.0000000|0   |0  |19 |19           |
+----------------+----+---+---+-------------+
only showing top 5 rows



In [8]:
logs_copy = logs.withColumn('duration_secs',
F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
F.col('Duration').substr(4,2).cast('int') * 60 +
F.col('Duration').substr(7,2).cast('int'))

In [ ]:
logs_copy.printSchema()

In [ ]:
logs_copy.select(F.col('Duration'), F.col('duration_secs')).show(5, False)

In [ ]:
logs_copy = logs_copy.select(*[x.lower() for x in logs_copy.columns])

In [15]:
logs_copy.toDF(*[x.lower() for x in logs_copy.columns]).printSchema()

root
 |-- broadcastlogid: integer (nullable = true)
 |-- logserviceid: integer (nullable = true)
 |-- logdate: timestamp (nullable = true)
 |-- sequenceno: integer (nullable = true)
 |-- audiencetargetageid: integer (nullable = true)
 |-- audiencetargetethnicid: integer (nullable = true)
 |-- categoryid: integer (nullable = true)
 |-- closedcaptionid: integer (nullable = true)
 |-- countryoforiginid: integer (nullable = true)
 |-- dubdramacreditid: integer (nullable = true)
 |-- ethnicprogramid: integer (nullable = true)
 |-- productionsourceid: integer (nullable = true)
 |-- programclassid: integer (nullable = true)
 |-- filmclassificationid: integer (nullable = true)
 |-- exhibitionid: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- endtime: string (nullable = true)
 |-- logentrydate: timestamp (nullable = true)
 |-- productionno: string (nullable = true)
 |-- programtitle: string (nullable = true)
 |-- starttime: string (nullable = true)
 |-- subtitle: string 

In [ ]:
logs_copy.columns

In [ ]:
for w in logs_copy.columns:
    logs_copy.describe(w).show()

In [ ]:
logs_copy.columns